In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycaret4
from pycaret.classification import *
from pycaret.datasets import get_data

In [ ]:
import pandas as pd
from pycaret.regression import *

train_data = pd.read_csv('train.csv')

train_data['DATETIMEDATA'] = pd.to_datetime(train_data['DATETIMEDATA'])

regression_setup = setup(train_data, target='PM25', session_id=123, date_features=['DATETIMEDATA'], normalize=True)
best = compare_models()

rf_model = create_model('rf')

tuned_rf_model = tune_model(rf_model)

bagged_model = ensemble_model(tuned_rf_model, n_estimators=20)

last_prediction_time = train_data['DATETIMEDATA'].max()

start_date = last_prediction_time + pd.DateOffset(hours=1)
end_date = start_date + pd.DateOffset(days=30, hours=23)
next_week_hours = pd.date_range(start=start_date, end=end_date, freq='D')

next_week_data = pd.DataFrame({'DATETIMEDATA': next_week_hours})

next_week_data['O3'] = 0
next_week_data['WS'] = 0
next_week_data['TEMP'] = 0
next_week_data['RH'] = 0
next_week_data['WD'] = 0

model_predictions = predict_model(bagged_model, data=next_week_data)

mpdf = pd.DataFrame(model_predictions, columns=['DATETIMEDATA', 'prediction_label'])

print(mpdf)